# 02450 Project 2


The primary goal of Project 2 is to apply supervised learning techniques (regression and classification) to predict properties or classifications of wood based on the cleaned and scaled dataset from Project 1.

In [57]:
import os
import numpy as np
import glob
import PIL.Image as Image
from tqdm.notebook import tqdm
import xlrd
import scipy
from scipy.linalg import svd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import (
    figure,
    grid,
    legend,
    loglog,
    plot,
    semilogx,
    show,
    subplot,
    title,
    xlabel,
    ylabel,
)
from sklearn.naive_bayes import BernoulliNB  # Use BernoulliNB for binary classification
from sklearn.model_selection import KFold
import sklearn.linear_model as lm
from matplotlib.pylab import figure, hist, plot, show, subplot, xlabel, ylabel
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from dtuimldmtools import *
from dtuimldmtools.statistics.statistics import correlated_ttest
from dtuimldmtools import confmatplot, rocplot
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.model_selection import KFold

### Cleaned and Scaled Data from Project 1

Hardwood= 1, Softwood = 0

In [58]:
cwd = os.getcwd()
data = pd.read_csv(cwd + '/usa_wood_data_formatted.csv')

removed_columns = ['side_hardness','tension_perpendicular_to_grain','impact_bending']
data_cleaned = data.drop(columns=removed_columns)
data_cleaned = data_cleaned.dropna()
print(data_cleaned.isnull().sum()) # 0 means data is clean

data_arr = np.array(data_cleaned)  # Attributes are columns
attributeNames = [name for name in data_cleaned.columns]
attributeNames = attributeNames[7:12]
attributeNames = np.concatenate((['Wood Type', 'Moisture Content'], attributeNames))
print(attributeNames)

for i in range(data_arr.shape[1]):
    val, count = np.unique(data_arr[:,i], return_counts=True)
    #print(val.shape)        # Print how many unique values of each attribute exist 
                            # Probably dont do one of k coding for first 4 attributes since they have so many unique values

data_adj = data_arr[:,6:13]
print("1",data_adj.shape)

for i in range(2):
    OoK = np.zeros([data_arr.shape[0], 1])
    val, count = np.unique(data_arr[:,5 - i], return_counts=True)
    
    for j in range(data_arr.shape[0]):
        if data_arr[j,5-i] == val[0]:
            OoK[j, 0] = 1
    data_adj = np.concatenate((OoK, data_adj), 1)
print("2",data_adj.shape)


#classsification
y = data_adj[:,0]
X = data_adj[:,1:9]
#regression
#y = data_adj[:,4]
# = np.concatenate((data_adj[:,0:3], data_adj[:,5:]), 1)
N, M = X.shape

X = X.astype(np.float64)  # Convert to float32 (or float64 if necessary)
y = y.astype(np.float64)  # Convert y to float32

#print(y)
print("X-shape", X.shape)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)  # Features as float32
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1)  # Target variable as float32 and reshaped to (N, 1)
#X_tensor = torch.FloatTensor(X)
#y_tensor = torch.FloatTensor(X_scaled[:,6])
#y_tensor = torch.FloatTensor(y).unsqueeze(1)
#numpy.float64

common_species_name                   0
genus                                 0
species                               0
scientific_name                       0
classification                        0
moisture_content                      0
specific_gravity                      0
modulus_of_rupture                    0
modulus_of_elasticity                 0
work_to_maximum_load                  0
compression_parallel_to_grain         0
compression_perpendicular_to_grain    0
shear_parallel_to_grain               0
dtype: int64
['Wood Type' 'Moisture Content' 'modulus_of_rupture'
 'modulus_of_elasticity' 'work_to_maximum_load'
 'compression_parallel_to_grain' 'compression_perpendicular_to_grain']
1 (214, 7)
2 (214, 9)
X-shape (214, 8)


In [59]:
#K = 10
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, stratify=y)
def logistic_regression(X_train, y_train, X_test, y_test, l):
    
    # Standardize the training and test set based on training set mean and std
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis=0)
    X_train = (X_train - mu) / sigma
    X_test = (X_test - mu) / sigma

    # Set regularization strength with C (inverse of lambda)
    mdl = LogisticRegression(penalty="l2", C=1 / l, max_iter=1000)
    mdl.fit(X_train, y_train)

    # Predictions on training and test sets
    y_train_est = mdl.predict(X_train)
    y_test_est = mdl.predict(X_test)

    # Calculate error rates
    train_error_rate = np.mean(y_train_est != y_train)
    test_error_rate = np.mean(y_test_est != y_test)

    # Calculate coefficient norm
    coefficient_norm = np.linalg.norm(mdl.coef_)

    # Print results for this lambda
   # print(f"\nLambda = {l} (C = {1 / l}):")
    #print("Train Error Rate:", train_error_rate)
    #print("Test Error Rate:", test_error_rate)
   # print("Coefficient Norm:", coefficient_norm)
    # Return values for further use
    return test_error_rate #, coefficient_norm,  l, train_error_rate,

#l = 0.01
#lambda_value = 0.01  # Define a single lambda value
#logistic_regression(X_train, y_train, X_test, y_test, lambda_value)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.80, stratify=y)
def naive_classifier_binary(X_train, y_train, X_test, y_test, alpha, fit_prior=True):
    # Standardize the training and test set based on training set mean and std
    mu = np.mean(X_train, axis=0)
    sigma = np.std(X_train, axis=0)
    X_train = (X_train - mu) / sigma
    X_test = (X_test - mu) / sigma
    
    # Initialize and fit BernoulliNB for binary classification with the given alpha
    nb_classifier = BernoulliNB(alpha=alpha, fit_prior=fit_prior)
    nb_classifier.fit(X_train, y_train)
    
    # Predictions for test and train sets
    y_train_est = nb_classifier.predict(X_train)
    y_test_est = nb_classifier.predict(X_test)
    
    # Calculate error rates
    train_error = np.mean(y_train_est != y_train)
    test_error = np.mean(y_test_est != y_test)
    
    #print(f"Alpha: {alpha} | Train Error Rate: {train_error * 100:.2f}% | Test Error Rate: {test_error * 100:.2f}%")
    
    return test_error #alpha, train_error,

# Example usage:
alpha = 1.0  # Set a specific alpha value
naive_classifier_binary(X_train, y_train, X_test, y_test, alpha)

0.3313953488372093

In [61]:
#Baseline, based on splitting data before, but doing standardization in this split
#K = 10
#CV = model_selection.KFold(n_splits=K, shuffle=True)
#K = 10
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, stratify=y)
def baseline_classification(y_train,y_test):  
    error_rates = []
   
    # Find the most frequent class in the training labels
    unique_classes, counts = np.unique(y_train, return_counts=True)
    most_frequent_class = unique_classes[np.argmax(counts)]
    #print("The most frequent class is:", most_frequent_class)

    #Create predictions for the test set using the most frequent class
    y_pred_baseline = np.full(y_test.shape, most_frequent_class)  # predict the most frequent class for all

    #Evaluate the baseline model
    baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
    baseline_error = 1 - baseline_accuracy
    error_rates.append(baseline_error)

    # Calculate the average test error rate across all folds
    avg_test_error_rate = np.mean(error_rates)
    #print(f"Average Test Error Rate: {avg_test_error_rate:.2f}")
    #print(f"Baseline Accuracy: {baseline_accuracy:.2f}")
    return avg_test_error_rate
#baseline_classification(y_train,y_test)
#baseline_classification(y_train,y_test)

In [65]:
from sklearn.model_selection import KFold
import numpy as np

# Model parameters
#lambdas = np.logspace(-2, 0, 50)  # From 10^-2 to 10^0 (0.01 to 1) with 50 points
#alphas = np.logspace(-2, 0, 50)
#alphas = [0.1, 0.5, 1.0, 2.0, 5.0]
#number_of_logreg_models = len(lambdas)
#number_of_alphas = len(alphas)

def two_factor_cross_validation(X, y, alphas, lambdas, K1=10, K2=10):
    # Model parameters
    outer_cv = KFold(n_splits=K1, shuffle=True, random_state=42)
    inner_cv = KFold(n_splits=K2, shuffle=True, random_state=42)
    # Model parameters
    lambdas = np.logspace(-2, 0, 50)  # From 10^-2 to 10^0 (0.01 to 1) with 50 points
    alphas = np.logspace(-2, 2, 50)
    #alphas = [0.1, 0.5, 1.0, 2.0, 5.0]
    number_of_logreg_models = len(lambdas)
    number_of_alphas = len(alphas)
    
    # Store results for each model
    results_matrix = np.zeros((K1, 6), dtype=object)
    r_results_matrix = np.zeros((K1, 6), dtype=object)
    outer_error_array = np.zeros(K1)
    selected_models = np.zeros(K1, dtype=int)  # To store the index of the selected model
    selected_alphas = np.zeros(K1)  # To store selected alphas for Naive Bayes

    # Set numpy print options to four decimal places
    #np.set_printoptions(formatter={'float': '{:0.4f}'.format})
    outer_fold = 0
    for outer_train_index, outer_test_index in outer_cv.split(X):
        X_train_outer, X_test_outer = X[outer_train_index], X[outer_test_index]
        y_train_outer, y_test_outer = y[outer_train_index], y[outer_test_index]

        # Inner cross-validation loop
        inner_error_matrix = np.zeros((K2, number_of_logreg_models + number_of_alphas + 1))
        inner_genE_array = np.zeros(number_of_logreg_models + number_of_alphas + 1)

        inner_fold = 0
        for inner_train_index, inner_val_index in inner_cv.split(X_train_outer):
            X_train_inner, X_val_inner = X_train_outer[inner_train_index], X_train_outer[inner_val_index]
            y_train_inner, y_val_inner = y_train_outer[inner_train_index], y_train_outer[inner_val_index]

            # Logistic regression errors
            for l in range(len(lambdas)):
                inner_error_matrix[inner_fold, l] = logistic_regression(X_train_inner, y_train_inner, X_val_inner, y_val_inner, lambdas[l])
                #print("LogReg Nr. "+str(l)+" error: "+str(inner_error_matrix[inner_fold, l]))

            # Naive classifier errors
            for alph in range(len(alphas)):
                inner_error_matrix[inner_fold, number_of_logreg_models + alph] = naive_classifier_binary(X_train_inner, y_train_inner, X_val_inner, y_val_inner, alphas[alph])
                #print("Alpha Nr. "+str(alph)+" error: "+str(inner_error_matrix[inner_fold + number_of_logreg_models,alph])) #+ nr_of_logreg_models or number_of_alphas?
                #print("Alpha Nr. "+str(alph)+" error: "+str(inner_error_matrix[inner_fold, number_of_logreg_models + alph]))

            # Baseline error
            inner_error_matrix[inner_fold, -1] = baseline_classification(y_train_outer, y_test_outer)
            #print("Baseline error: "+str(inner_error_matrix[inner_fold, -1]))
            #print("Inner fold "+str(inner_fold)+" done")
            inner_fold += 1

        # Select the best model based on the minimum error
        for index in range(len(inner_genE_array)):
            inner_genE_array[index] = inner_error_matrix.T[index].mean()

        # Model selection logic: finding the best model
        selected_model = np.argmin(inner_genE_array)
        #print(f"Best model for outer fold {outer_fold}: Model {selected_model} with error {inner_genE_array[selected_model]}")
        
        # Store the selected model index
        selected_models[outer_fold] = selected_model

        # If Logistic Regression, store the lambda value
        if selected_model < number_of_logreg_models:
            selected_lambdas = lambdas[selected_model]
            logistic_error = logistic_regression(X_train_outer, y_train_outer, X_test_outer, y_test_outer, selected_lambdas)
            results_matrix[outer_fold, 3] = selected_lambdas  # Lambda for Logistic Regression
            results_matrix[outer_fold, 4] = logistic_error  # Logistic Regression error
        else:
            logistic_error = None
        #print(selected_model)
        #print(number_of_logreg_models)
        #print(number_of_logreg_models + number_of_alphas)
        # If Naive Bayes, store the alpha value
        if selected_model < number_of_logreg_models:
        #if selected_model >= number_of_logreg_models and selected_model < (number_of_logreg_models + number_of_alphas):
            selected_alpha = alphas[selected_model - number_of_logreg_models]
            naive_error = naive_classifier_binary(X_train_outer, y_train_outer, X_test_outer, y_test_outer, selected_alpha)
            results_matrix[outer_fold, 1] = selected_alpha  # Alpha for Naive Bayes
            results_matrix[outer_fold, 2] = naive_error  # Naive Bayes error
        else:
            naive_error = None
        #print("selected model",selected_model)
        # Evaluate the baseline classifier on the outer test set
        baseline_error = baseline_classification(y_train_outer, y_test_outer)
        results_matrix[outer_fold, 5] = baseline_error  # Baseline error

        # Store the results for the fold
        results_matrix[outer_fold, 0] = outer_fold + 1  # Fold number

        #print("Outer fold "+str(outer_fold) +" done with errors:")
        #print(f"Logistic Regression error: {logistic_error}, Naive Bayes error: {naive_error}, Baseline error: {baseline_error}")
        outer_fold += 1
    #final_matrix = np.vstack(results_matrix)
    #print(final_matrix)
    # Print each element in the matrix with four decimal places
    for row in results_matrix:
        print(" ".join(f"{elem:.4f}" for elem in row))
    # Convert the matrix to a DataFrame
    #print("Matrix saved as 'results_matrix.xlsx'")
    return results_matrix
     
np.savetxt("data3.csv", results_matrix, delimiter = ",")
folder_path = "/Users/selmakuhlmann/Documents/WS 2024/MachineLearning/Python/MachLearn/Project 2/02450-Project-2"  # Replace with your desired folder path
# Ensure the folder exists
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, "data3.csv")
np.savetxt(file_path, results_matrix, delimiter=",", fmt="%.4f")  # '%.4f' formats to 4 decimal places
print(f"Matrix saved as 'data3.csv' in {folder_path}")         
   
    # Display final results matrix
    #print("\nResults Matrix (10x6):\n", results_matrix)
 #   return results_matrix

# Call the function to perform the two-factor cross-validation
two_factor_cross_validation(X, y, alphas, lambdas, K1=10, K2=10)

#np.save("results_matrix.npy", results_matrix)

Matrix saved as 'data3.csv' in /Users/selmakuhlmann/Documents/WS 2024/MachineLearning/Python/MachLearn/Project 2/02450-Project-2
1.0000 22.2300 0.3636 0.4715 0.0455 0.5000
2.0000 18.4207 0.1818 0.4292 0.1818 0.4545
3.0000 26.8270 0.1818 0.5179 0.0455 0.4545
4.0000 7.1969 0.3182 0.2683 0.1818 0.2727
5.0000 0.0954 0.3333 0.0309 0.2857 0.2857
6.0000 10.4811 0.4286 0.3237 0.1905 0.4762
7.0000 12.6486 0.0952 0.3556 0.1905 0.4286
8.0000 0.0791 0.2857 0.0281 0.1905 0.3810
9.0000 4.9417 0.1429 0.2223 0.0952 0.5238
10.0000 100.0000 0.3333 1.0000 0.2381 0.3333


array([[1, 22.229964825261934, 0.36363636363636365, 0.47148663634573923,
        0.045454545454545456, 0.5],
       [2, 18.420699693267146, 0.18181818181818182, 0.42919342601287763,
        0.18181818181818182, 0.4545454545454546],
       [3, 26.826957952797247, 0.18181818181818182, 0.517947467923121,
        0.045454545454545456, 0.4545454545454546],
       [4, 7.196856730011514, 0.3181818181818182, 0.2682695795279725,
        0.18181818181818182, 0.2727272727272727],
       [5, 0.09540954763499938, 0.3333333333333333, 0.030888435964774818,
        0.2857142857142857, 0.2857142857142857],
       [6, 10.481131341546853, 0.42857142857142855, 0.32374575428176433,
        0.19047619047619047, 0.47619047619047616],
       [7, 12.648552168552959, 0.09523809523809523, 0.35564803062231287,
        0.19047619047619047, 0.4285714285714286],
       [8, 0.07906043210907697, 0.2857142857142857, 0.028117686979742307,
        0.19047619047619047, 0.38095238095238093],
       [9, 4.941713361323833, 0

In [68]:
#cwd = os.getcwd()
#data = pd.read_csv(cwd + '/data3.csv')
# Load the saved results_matrix if needed
#results_matrix = np.load("results_matrix.npy")
#results_matrix = two_factor_cross_validation(X, y, alphas, lambdas, K1=10, K2=10)
#print(results_matrix)
from dtuimldmtools import mcnemar

alpha = 0.05  # Significance level for the test

# Perform McNemar's test for each pair of columns
# Compare column 2 (Naive Bayes) and column 4 (Logistic Regression)
thetahat_2_4, CI_2_4, p_2_4 = mcnemar(results_matrix[:, 2], results_matrix[:, 4], alpha=alpha)
print("Comparison between columns 2 and 4 (Naive Bayes vs Logistic Regression):")
print("theta =", thetahat_2_4, " CI:", CI_2_4, "p-value:", p_2_4)

# Compare column 2 (Naive Bayes) and column 5 (Baseline)
thetahat_2_5, CI_2_5, p_2_5 = mcnemar(results_matrix[:, 2], results_matrix[:, 5], alpha=alpha)
print("\nComparison between columns 2 and 5 (Naive Bayes vs Baseline):")
print("theta =", thetahat_2_5, " CI:", CI_2_5, "p-value:", p_2_5)

# Compare column 4 (Logistic Regression) and column 5 (Baseline)
thetahat_4_5, CI_4_5, p_4_5 = mcnemar(results_matrix[:, 4], results_matrix[:, 5], alpha=alpha)
print("\nComparison between columns 4 and 5 (Logistic Regression vs Baseline):")
print("theta =", thetahat_4_5, " CI:", CI_4_5, "p-value:", p_4_5)

[[1 0.47148663634573923 0.2727272727272727 0.47148663634573923
  0.045454545454545456 0.5]
 [2 0.42919342601287763 0.18181818181818182 0.42919342601287763
  0.18181818181818182 0.4545454545454546]
 [3 0.517947467923121 0.13636363636363635 0.517947467923121
  0.045454545454545456 0.4545454545454546]
 [4 0.2682695795279725 0.3181818181818182 0.2682695795279725
  0.18181818181818182 0.2727272727272727]
 [5 0.030888435964774818 0.3333333333333333 0.030888435964774818
  0.2857142857142857 0.2857142857142857]
 [6 0.32374575428176433 0.42857142857142855 0.32374575428176433
  0.19047619047619047 0.47619047619047616]
 [7 0.35564803062231287 0.14285714285714285 0.35564803062231287
  0.19047619047619047 0.4285714285714286]
 [8 0.028117686979742307 0.2857142857142857 0.028117686979742307
  0.19047619047619047 0.38095238095238093]
 [9 0.22229964825261944 0.14285714285714285 0.22229964825261944
  0.09523809523809523 0.5238095238095238]
 [10 1.0 0.3333333333333333 1.0 0.23809523809523808 0.3333333333

TypeError: mcnemar() missing 1 required positional argument: 'yhatB'

In [74]:
#Optimal_lambda = result_matrix [:,4]
print("Sum",sum(results_matrix [:,:]))
print("Mean",(sum(results_matrix [:,:])/10))
#optimal lambda: 0.36475966659109244, but probably wrong

Sum [55 3.6475966659109242 2.5757575757575757 3.6475966659109242
 1.6450216450216448 4.1103896103896105]
Mean [5.5 0.36475966659109244 0.25757575757575757 0.36475966659109244
 0.16450216450216448 0.41103896103896104]
